In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from numpy import mean,std
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,recall_score
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.naive_bayes import GaussianNB

## Loading Data

In [4]:
data = data = pd.read_csv('../Datasets/telecommunications_churn.csv',sep=';')
data.head()

,account_length,voice_mail_plan,voice_mail_messages,day_mins,evening_mins,night_mins,international_mins,customer_service_calls,international_plan,day_calls,day_charge,evening_calls,evening_charge,night_calls,night_charge,international_calls,international_charge,total_charge,churn
0,128,1,25,265.1,197.4,244.7,10.0,1,0,110,45.07,99,16.78,91,11.01,3,2.70,75.56,0
1,107,1,26,161.6,195.5,254.4,13.7,1,0,123,27.47,103,16.62,103,11.45,3,3.70,59.24,0
2,137,0,0,243.4,121.2,162.6,12.2,0,0,114,41.38,110,10.30,104,7.32,5,3.29,62.29,0
3,84,0,0,299.4,61.9,196.9,6.6,2,1,71,50.90,88,5.26,89,8.86,7,1.78,66.80,0
4,75,0,0,166.7,148.3,186.9,10.1,3,1,113,28.34,122,12.61,121,8.41,3,2.73,52.09,0


## Scaling Data and Seperating X and y

In [5]:
std_scr = StandardScaler()
X = data.drop(labels='churn',axis=1)
cols = X.columns
X = pd.DataFrame(std_scr.fit_transform(X))
X.columns = cols
y = data[['churn']]
X.shape,y.shape

((3333, 18), (3333, 1))

## Train Test Split

In [6]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=12,stratify=y)

### Dict to record model performance

In [8]:
my_dict = {'model':[], 'accuracy': [],'precision':[],'recall':[]}

### Function to pridict and record model performance

In [9]:
def populate_matrix(fittedModel):
    y_pred = fittedModel.predict(x_test)
    accuracy = round(accuracy_score(y_test,y_pred)*100,2)
    precision = round(precision_score(y_test,y_pred)*100,2)
    recall = round(recall_score(y_test,y_pred)*100,2)
    model = str(fittedModel)
    my_dict["model"].append(model)
    my_dict["accuracy"].append(accuracy)
    my_dict["precision"].append(precision)
    my_dict["recall"].append(recall)


## Model Validation

## Logistic Model

In [10]:
# evaluate model
logistic = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=logistic,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Accuracy: 0.861 (0.009)


### Model Training

In [11]:

logistic.fit(x_train,y_train)
populate_matrix(logistic)
print(my_dict)

{'model': ['LogisticRegression()'], 'accuracy': [86.51], 'precision': [56.6], 'recall': [30.93]}


## Decision Tree

In [12]:
dtree = DecisionTreeClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=dtree,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.950 (0.009)


### Model Training

In [13]:
dtree.fit(x_train,y_train)
populate_matrix(dtree)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()'], 'accuracy': [86.51, 94.0], 'precision': [56.6, 76.15], 'recall': [30.93, 85.57]}


## RandomForest

In [14]:
rf = RandomForestClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=rf,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.978 (0.008)


### Model Training

In [15]:
rf.fit(x_train,y_train)
populate_matrix(rf)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()'], 'accuracy': [86.51, 94.0, 97.45], 'precision': [56.6, 76.15, 100.0], 'recall': [30.93, 85.57, 82.47]}


## Gradient Boosting

In [16]:
gboost = GradientBoostingClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=gboost,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.979 (0.007)


### Model Training

In [17]:
gboost.fit(x_train,y_train)
populate_matrix(gboost)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()'], 'accuracy': [86.51, 94.0, 97.45, 97.6], 'precision': [56.6, 76.15, 100.0, 97.65], 'recall': [30.93, 85.57, 82.47, 85.57]}


## SVM

In [18]:
svm = SVC()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=svm,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.928 (0.011)


### Model Training

In [19]:
svm.fit(x_train,y_train)
populate_matrix(svm)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()'], 'accuracy': [86.51, 94.0, 97.45, 97.6, 92.8], 'precision': [56.6, 76.15, 100.0, 97.65, 88.89], 'recall': [30.93, 85.57, 82.47, 85.57, 57.73]}


## ANN

In [20]:
ann_model = Sequential()
ann_model.add(Dropout(0.30))
ann_model.add(Dense(20,input_dim=x_train.shape[1],activation='relu'))
ann_model.add(Dense(10,activation='relu'))
ann_model.add(Dropout(0.30))
ann_model.add(Dense(5,activation='relu'))
ann_model.add(Dense(1,activation='sigmoid'))


In [21]:
ann_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


### Model Training

In [22]:
ann_model.fit(x_train,y_train,batch_size=100,epochs=200,validation_data=(x_test,y_test),shuffle=True)



Epoch 1/200
27/27 [==============================] - 1s 7ms/step - loss: 0.6049 - accuracy: 0.7644 - val_loss: 0.5534 - val_accuracy: 0.8486
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5390 - accuracy: 0.8417 - val_loss: 0.4833 - val_accuracy: 0.8546
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4889 - accuracy: 0.8533 - val_loss: 0.4381 - val_accuracy: 0.8546
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4562 - accuracy: 0.8560 - val_loss: 0.4092 - val_accuracy: 0.8546
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4529 - accuracy: 0.8571 - val_loss: 0.3899 - val_accuracy: 0.8561
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.8567 - val_loss: 0.3737 - val_accuracy: 0.8561
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 0.4128 - accuracy: 0.8601 - val_loss: 0.3622 - val_accuracy: 0.8591
Epoch 8/200
2

Epoch 59/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3096 - accuracy: 0.8852 - val_loss: 0.2620 - val_accuracy: 0.8996
Epoch 60/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3069 - accuracy: 0.8867 - val_loss: 0.2647 - val_accuracy: 0.8981
Epoch 61/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3101 - accuracy: 0.8852 - val_loss: 0.2647 - val_accuracy: 0.8966
Epoch 62/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3052 - accuracy: 0.8927 - val_loss: 0.2631 - val_accuracy: 0.8981
Epoch 63/200
27/27 [==============================] - 0s 1ms/step - loss: 0.3001 - accuracy: 0.8927 - val_loss: 0.2599 - val_accuracy: 0.9010
Epoch 64/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2916 - accuracy: 0.8912 - val_loss: 0.2563 - val_accuracy: 0.8996
Epoch 65/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3060 - accuracy: 0.8942 - val_loss: 0.2586 - val_accuracy: 0.8996
Epoch 

27/27 [==============================] - 0s 2ms/step - loss: 0.2618 - accuracy: 0.9141 - val_loss: 0.2650 - val_accuracy: 0.9025
Epoch 174/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2531 - accuracy: 0.9152 - val_loss: 0.2634 - val_accuracy: 0.9040
Epoch 175/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2476 - accuracy: 0.9167 - val_loss: 0.2551 - val_accuracy: 0.9055
Epoch 176/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2368 - accuracy: 0.9224 - val_loss: 0.2598 - val_accuracy: 0.9055
Epoch 177/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2541 - accuracy: 0.9160 - val_loss: 0.2637 - val_accuracy: 0.9040
Epoch 178/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2480 - accuracy: 0.9149 - val_loss: 0.2637 - val_accuracy: 0.9040
Epoch 179/200
27/27 [==============================] - 0s 2ms/step - loss: 0.2630 - accuracy: 0.9122 - val_loss: 0.2645 - val_accuracy: 0.9010
Epoch 180/200

### convertiing probabilities to labels

In [23]:
y_pred = ann_model.predict(x_test)
for i in range(0,len(y_pred)):
    if y_pred[i]>=.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0
   

In [24]:
my_dict["model"].append('ANN')
my_dict["accuracy"].append(round(accuracy_score(y_test,y_pred)*100,2))
my_dict["precision"].append(round(precision_score(y_test,y_pred)*100,2))
my_dict["recall"].append(round(recall_score(y_test,y_pred)*100,2))

### Naive Bayes

In [25]:
gnb = GaussianNB()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=gnb,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.875 (0.017)


In [26]:
gnb.fit(x_train,y_train)
populate_matrix(gnb)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()', 'ANN', 'GaussianNB()'], 'accuracy': [86.51, 94.0, 97.45, 97.6, 92.8, 90.4, 88.31], 'precision': [56.6, 76.15, 100.0, 97.65, 88.89, 92.31, 57.6], 'recall': [30.93, 85.57, 82.47, 85.57, 57.73, 37.11, 74.23]}


### Final performance matrix of all the models

In [27]:
pd.DataFrame(my_dict)

,model,accuracy,precision,recall
0,LogisticRegression(),86.51,56.60,30.93
1,DecisionTreeClassifier(),94.00,76.15,85.57
2,RandomForestClassifier(),97.45,100.00,82.47
3,GradientBoostingClassifier(),97.60,97.65,85.57
4,SVC(),92.80,88.89,57.73
5,ANN,90.40,92.31,37.11
6,GaussianNB(),88.31,57.60,74.23
